# Hyperparameter tuning

In de reeds besproken machine learning technieken hebben we reeds een aantal keer vermeld dat er hyperparameters (denk aan regularisatieparameters, manieren van regularisatie, kernel type, ...).

In het geval van lineaire regressie gaat het dan over:
* L1 of L2 norm
* Regularisatieparameter $\lambda$
* learning rate

In het geval van SVM over:
* Type kernel
* Regularisatieparameter C
* Regularisatieparameter $\gamma$

Tot nu bestond de zoektocht naar de optimale combinatie van deze parameters door het manueel uitproberen en evalueren van een reeks combinaties van parameters.
Deze methode is echter niet schaalbaar en kan geautomatiseerd worden.
Dit gebeurd door middel van een gridsearch.

## Gridsearch

Het gridsearch algoritme bestaat eruit om een lijst op te stellen voor elke parameter welke waarden moeten getest worden.
Voor elke mogelijke combinatie van parameters gaat er dan een model getrained en geevalueerd worden.
Een voorbeeld van hoe dit kan geautomatiseerd worden binnen sklearn kan [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html#sklearn.model_selection.GridSearchCV) gevonden worden.

In [6]:
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
# halving moet enabled worden want is nog experimenteel.
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import HalvingGridSearchCV
import numpy as np

iris = datasets.load_iris()
# kies de parameters die getest worden (dit kan een dictionary zijn)
parameters = {'kernel':('linear', 'rbf'), 'C':np.arange(1, 10, 1), "gamma": np.arange(0.1, 10, 0.1)}

svc = svm.SVC()
clf = HalvingGridSearchCV(svc, parameters)
%time clf.fit(iris.data, iris.target)
clf.get_params()

Wall time: 15.4 s


{'aggressive_elimination': False,
 'cv': 5,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__break_ties': False,
 'estimator__cache_size': 200,
 'estimator__class_weight': None,
 'estimator__coef0': 0.0,
 'estimator__decision_function_shape': 'ovr',
 'estimator__degree': 3,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__probability': False,
 'estimator__random_state': None,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVC(),
 'factor': 3,
 'max_resources': 'auto',
 'min_resources': 'exhaust',
 'n_jobs': None,
 'param_grid': {'kernel': ('linear', 'rbf'),
  'C': array([1, 2, 3, 4, 5, 6, 7, 8, 9]),
  'gamma': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. , 1.1, 1.2, 1.3,
         1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2. , 2.1, 2.2, 2.3, 2.4, 2.5, 2.6,
         2.7, 2.8, 2.9, 3. , 3.1, 3.2, 3.3, 3.4, 3.5, 3.6, 3.7, 3.8, 3.9,
         4. , 4.1, 4.2, 4.3, 4.4, 4.5, 4.6, 4.7, 4

In [3]:
# rechtstreeks predict is mogelijk
clf.predict(iris.data)
# maar je kan ook het beste model selecteren
beste_model = clf.best_estimator_

{'C': 2, 'gamma': 0.2, 'kernel': 'rbf'}

De standaard methode van hierboven gaat alle combinaties afgaan.
Andere methoden die sneller maar niet alle combinaties aftoetsen zijn
* [RandomizedSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html#sklearn.model_selection.RandomizedSearchCV)
*[HalvingGridSearchCv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingGridSearchCV.html#sklearn.model_selection.HalvingGridSearchCV)
*[HalvingRandomizedSearchCv](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.HalvingRandomSearchCV.html#sklearn.model_selection.HalvingRandomSearchCV)

Belangrijk om hierbij op te merken is dat het GridSearch algoritme enkel verschillende parameters van het model trained en dat er geen eigenschappen van de data kan veranderd worden.
Indien je ook een exhausieve search wilt doen van het aantal hogere orde features of de vorm van scaling die gebruikt wordt op input parameters. Moet je een eigen wrapper schrijven die nog deze zaken uittest en de performantie van de uiteindelijke modellen vergelijkt.

## Validatieset

Welke data kunnen we nu gebruiken om deze gridsearch te evalueren.
Zowel de testdata als de trainingsdata kan niet gebruikt worden omdat we niet kunnen evalueren op de data waarmee het model getrained is.
Om deze reden wordt de dataset typisch in drie opgedeeld, namelijk een training-, test- en validatieset.
De validatieset is de data die dan gebruikt kan worden voor hyperparameter tuning.
Typisch wordt de dataset dan in de volgende groottes opgedeeld:
* Testset: 15%
* Validatieset: 15% 
* Trainingsdata: 70%

Dit zijn echter geen vaste waarden en kunnen wat verschillen in de praktijk.
Hoe meer data je beschibaar is hoe groter het percentage trainingsdata kan zijn. 
In het geval van big-data applicaties kan dit oplopen tot 98%.

## K-fold cross validation

Bij het steeds gebruiken van dezelfde validatieset is het mogelijk dat er een unieke split is die leidt tot een onverwacht goed of slecht resultaat.
Om dit tegen te gaan kan er gebruik gemaakt worden van K-fold cross validation.
Daarbij berekenen we de verwachte error K keer, elke keer met een andere train en validatie set om zo de kans te verhogen dat het uiteindelijke model ook goed werkt op de testset met ongeziene data.
Standaard wordt er bij het gebruik van het gridsearch algoritme gebruik gemaakt van 5 folds voor het zoeken naar de beste hyperparameters.
Indien de standaard manier niet voldoet voor de gewenste toepassing kan je ook de split rechtstreeks uitvoeren.
Meer informatie over deze methode vind je [hier](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html)

![kfold cross validation](images/kFold.png)

## Oefening

In [deze dataset](https://www.kaggle.com/mathchi/diabetes-data-set) is een hele reeks data beschikbaar over een aantal medische eigenschappen van personen en of deze personen diabetes hebben of niet.
Ga nu op zoek naar het beste model om te voorspellen of een persoon diabetes gaat hebben of niet.
Test hierbij zowel de logistische regressie en svm methoden en maak gebruik van gridsearch met 10-fold cross validation om de verschillende hyperparameters te testen. 

Wat is de hoogst behaalde accuraatheid en de benodigde hyperparameters?

Indien dit gelukt is, zoek ook het model dat de hoogste weighted f1-score behaald. 
Welke techniek gebruikte dit model en welke hyperparameters zijn er hiervoor gekozen?
Vergelijk beide modellen. Is er een significant verschil in de resulterende hyperparameters?
Is de behaalde accuraatheid sterk afwijkend?

In [7]:
import opendatasets as od
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC

In [8]:
#od.download("https://www.kaggle.com/mathchi/diabetes-data-set")

In [9]:
df = pd.read_csv("./diabetes-data-set/diabetes.csv")
display(df.head())
y = df.Outcome
X = df.drop("Outcome", axis=1).values

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
# waarom doen we hier geen fit? Omdat we het op dezelfde manier willen scalen als de train-set
X_test = scaler.transform(X_test)

#model = LogisticRegression()
model = SVC()
# ga naar de api en kijk wat we kunnen varieren
#parameters = {'penalty':['l1', 'l2'], 'C':np.arange(0.1, 10, 0.1), 'solver':['liblinear']}
#parameters = {'C':np.arange(0.1, 10, 0.1), 'solver':['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'], 'penalty':['l2']}
# list of dictionaries om combinaties van parameters te vermijden die niet toegelaten/nuttig zijn
parameters =  [{'C':np.arange(0.1, 10, 0.4), 'kernel':['linear']},
              {'C':np.arange(0.1, 10, 0.4), 'kernel':['rbf'], "gamma":np.arange(0.1, 1, 0.1)},
              {'C':np.arange(0.1, 10, 0.4), 'kernel':['linear'],"gamma":np.arange(0.1, 1, 0.1), "degree":np.arange(2, 4)}]

gridSearch = GridSearchCV(model, parameters, cv=10, verbose=2) # , scoring="f1_weighted"
gridSearch.fit(X_train, y_train)
gridSearch.get_params()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


Fitting 10 folds for each of 700 candidates, totalling 7000 fits
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=0.

[CV] END ...............................C=4.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=4.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=4.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=4.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=4.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=4.1, kernel=linear; total time=   0.0s
[CV] END ...............................C=4.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=4.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=4.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=4.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=4.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=4.5, kernel=linear; total time=   0.0s
[CV] END ...................

[CV] END ...............................C=8.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=8.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=8.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=8.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=8.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=8.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=8.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=8.5, kernel=linear; total time=   0.0s
[CV] END ...............................C=8.9, kernel=linear; total time=   0.0s
[CV] END ...............................C=8.9, kernel=linear; total time=   0.0s
[CV] END ...............................C=8.9, kernel=linear; total time=   0.0s
[CV] END ...............................C=8.9, kernel=linear; total time=   0.0s
[CV] END ...................

[CV] END .......................C=0.1, gamma=0.8, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.8, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.8, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.8, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.8, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.8, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.8, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.8, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.9, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.9, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.9, kernel=rbf; total time=   0.0s
[CV] END .......................C=0.1, gamma=0.9, kernel=rbf; total time=   0.0s
[CV] END ...................

...

[CV] END ..........C=8.9, degree=3, gamma=0.6, kernel=linear; total time=   0.0s
[CV] END ..........C=8.9, degree=3, gamma=0.6, kernel=linear; total time=   0.0s
[CV] END ..........C=8.9, degree=3, gamma=0.6, kernel=linear; total time=   0.0s
[CV] END ..........C=8.9, degree=3, gamma=0.6, kernel=linear; total time=   0.0s
[CV] END ..........C=8.9, degree=3, gamma=0.6, kernel=linear; total time=   0.0s
[CV] END ..........C=8.9, degree=3, gamma=0.6, kernel=linear; total time=   0.0s
[CV] END ..........C=8.9, degree=3, gamma=0.6, kernel=linear; total time=   0.0s
[CV] END ..........C=8.9, degree=3, gamma=0.6, kernel=linear; total time=   0.0s
[CV] END C=8.9, degree=3, gamma=0.7000000000000001, kernel=linear; total time=   0.0s
[CV] END C=8.9, degree=3, gamma=0.7000000000000001, kernel=linear; total time=   0.0s
[CV] END C=8.9, degree=3, gamma=0.7000000000000001, kernel=linear; total time=   0.0s
[CV] END C=8.9, degree=3, gamma=0.7000000000000001, kernel=linear; total time=   0.0s
[CV] END

[CV] END C=9.3, degree=2, gamma=0.7000000000000001, kernel=linear; total time=   0.0s
[CV] END C=9.3, degree=2, gamma=0.7000000000000001, kernel=linear; total time=   0.0s
[CV] END C=9.3, degree=2, gamma=0.7000000000000001, kernel=linear; total time=   0.0s
[CV] END C=9.3, degree=2, gamma=0.7000000000000001, kernel=linear; total time=   0.0s
[CV] END C=9.3, degree=2, gamma=0.7000000000000001, kernel=linear; total time=   0.0s
[CV] END C=9.3, degree=2, gamma=0.7000000000000001, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=2, gamma=0.8, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=2, gamma=0.8, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=2, gamma=0.8, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=2, gamma=0.8, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=2, gamma=0.8, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=2, gamma=0.8, kernel=linear; total time=   0.0

[CV] END ..........C=9.3, degree=3, gamma=0.9, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=3, gamma=0.9, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=3, gamma=0.9, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=3, gamma=0.9, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=3, gamma=0.9, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=3, gamma=0.9, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=3, gamma=0.9, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=3, gamma=0.9, kernel=linear; total time=   0.0s
[CV] END ..........C=9.3, degree=3, gamma=0.9, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=2, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=2, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=2, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700

[CV] END C=9.700000000000001, degree=2, gamma=0.9, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=2, gamma=0.9, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=3, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=3, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=3, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=3, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=3, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=3, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=3, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=3, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=3, gamma=0.1, kernel=linear; total time=   0.0s
[CV] END C=9.700000000000001, degree=3, gamma=0.1, kernel=linear;

{'cv': 10,
 'error_score': nan,
 'estimator__C': 1.0,
 'estimator__break_ties': False,
 'estimator__cache_size': 200,
 'estimator__class_weight': None,
 'estimator__coef0': 0.0,
 'estimator__decision_function_shape': 'ovr',
 'estimator__degree': 3,
 'estimator__gamma': 'scale',
 'estimator__kernel': 'rbf',
 'estimator__max_iter': -1,
 'estimator__probability': False,
 'estimator__random_state': None,
 'estimator__shrinking': True,
 'estimator__tol': 0.001,
 'estimator__verbose': False,
 'estimator': SVC(),
 'n_jobs': None,
 'param_grid': [{'C': array([0.1, 0.5, 0.9, 1.3, 1.7, 2.1, 2.5, 2.9, 3.3, 3.7, 4.1, 4.5, 4.9,
          5.3, 5.7, 6.1, 6.5, 6.9, 7.3, 7.7, 8.1, 8.5, 8.9, 9.3, 9.7]),
   'kernel': ['linear']},
  {'C': array([0.1, 0.5, 0.9, 1.3, 1.7, 2.1, 2.5, 2.9, 3.3, 3.7, 4.1, 4.5, 4.9,
          5.3, 5.7, 6.1, 6.5, 6.9, 7.3, 7.7, 8.1, 8.5, 8.9, 9.3, 9.7]),
   'kernel': ['rbf'],
   'gamma': array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])},
  {'C': array([0.1, 0.5, 0.9, 1.3, 1.7

In [12]:
gridSearch.best_params_
gridSearch.best_score_

0.7719725013220519

## Pipeline

Bij het werken met een gridsearch is er nog een probleem omtrent data leakage.
Omdat we in bovenstaande voorbeeld de scaling hebben gedaan op de volledige dataset is alle data in principe gebruikt om de scaling uit te voeren en dus gaat er steeds een effect zijn van de training-folds op de validation-fold wat niet gewenst is.
Dit kan onterecht de score verbeteren.

Om dit tegen te gaan kan men gebruik maken van Pipelines. Hiermee definieer je de flow die de data moet ondergaan om verwerkt te worden. De volledige pipeline of flow kan dan gegeven worden aan de gridsearch algoritme. Hierdoor worden de preprocessing stappen (zoals scalers, ordinal of one-hotencoder) steeds enkel op de training folds uitgevoerd.
Dit zorgt ervoor dat er geen data leakage kan optreden. 
Het bijkomende voordeel is dat ook parameters van de preprocessing stappen meegenomen kunnen worden in de gridsearch.

Let op dat NaN waarden invullen kan gebeuren in de pipeline maar rijen of kolommen weglaten kan niet gedaan worden tijdens de pipeline. Echter is het gebruik van een pipeline een goede stap in het verhogen van de leesbaarheid van een code stuk en wordt het daardoor veelvuldig gebruikt.

In [14]:
import numpy as np

from sklearn.compose import ColumnTransformer
from sklearn.datasets import fetch_openml
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV

np.random.seed(0)

X, y = fetch_openml("titanic", version=1, as_frame=True, return_X_y=True)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=0)

In [ ]:
numeric_features = ['age', 'fare']
# numerieke waarden hebben twee stappen nodig -> pipeline
# hier null-waarden invullen en dan scaling toepassen
# imputer in plaats van fillna
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

categorical_features = ['embarked', 'sex', 'pclass']
categorical_transformer = OneHotEncoder(handle_unknown='ignore')

# column transformer om kolommen anders te gaan behandelen
# een transformer bestaat uit drie zaken: een naam, de transformer/model, de kolommen waarop het moet toegepast worden.
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)])

# Append classifier to preprocessing pipeline. (clf is classifier)
# Now we have a full prediction pipeline.
# stap 1
clf = Pipeline(steps=[('preprocessor', preprocessor),
                      ('classifier', LogisticRegression())])

clf.fit(X_train, y_train)
print("model score: %.3f" % clf.score(X_test, y_test))

In [ ]:
# 2 underscores om in de pipeline hyperparameters op lagere niveaus aan te passen
param_grid = [{
    'preprocessor__num__imputer__strategy': ['mean', 'median'],
    'classifier__C': [0.1, 1.0, 10, 100],
},
{
    'preprocessor__num__imputer__strategy': ['constant'],
    'preprocessor__num__imputer__fill_value': [-1, -2],
    'classifier__C': [0.1, 1.0, 10, 100]    
}]

grid_search = GridSearchCV(clf, param_grid, cv=10)
grid_search

## Oefening

Maak een pipeline voor een gridsearch uit te voeren op de diabetes-dataset in sklearn. Gebruik een Min-Max scaler voor de numerieke waarden en een ordinal encoder voor de categorieke kolommen.
Zoek naar de beste combinatie van hyperparameters van een SVM-classifier door middel van een grid search met cross validation.